In [17]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 

In [18]:
#FIBVID Dataset
fibvidNews = pd.read_csv("FIBVID\FIBVID news.csv")
fibvidTweets = pd.read_csv("FIBVID\FIBVID origin tweet.csv")

fibvidNews = fibvidNews[["text","group"]]
fibvidTweets = fibvidTweets[["post_text","group"]]

fibvidNews = fibvidNews.rename(columns={"group":"label","text":"content"})
fibvidTweets = fibvidTweets.rename(columns={"group":"label","post_text":"content"})

fibvid1 = pd.concat([fibvidNews,fibvidTweets], ignore_index=True)

fibvid = fibvid1[(fibvid1.label == 0) | (fibvid1.label ==1)]

fibvid["label"] = fibvid["label"].replace(0,"real")
fibvid["label"] = fibvid["label"].replace(1,"fake")

fibvid.reset_index(drop=True)

content label
0     the (covid 19) cases are going up but it's bec...  fake
1     every election year has a disease  sars in 200...  fake
2     if you paid rent in march or april they got to...  fake
3              dr fauci: no reason to be wearing a mask  fake
4       drinking boiled garlic water will cure covid 19  fake
...                                                 ...   ...
1935  Struggling after speaking with ⁦@donlemon⁩ ton...  fake
1936  As the dust settles on Democrats' downballot f...  fake
1937  We broke the news on #OutLoudWithGiannoCaldwel...  fake
1938  I see your tweets and I’m trying to double and...  fake
1939  A 2020 peer-reviewed study published in the jo...  fake

[1940 rows x 2 columns]

In [3]:
# Rumour Dataset
rumour = pd.read_csv("Rumour Dataset\Rumour Dataset.txt")

rumour = rumour[["content","label"]]
rumour = rumour[(rumour.label == "T") | (rumour.label =="F")]
rumour["label"] = rumour["label"].replace("T","real")
rumour["label"] = rumour["label"].replace("F","fake")

rumour.reset_index(drop=True)

content label
0     Coronavirus was created in a government lab as...  fake
1     The lie that coronavirus came from a bat or a ...  fake
2     The health experts had predicted the virus cou...  fake
3     A video clip supposedly showed that the expone...  fake
4     Almost 200 people in Italy died from the coron...  real
...                                                 ...   ...
5554  “DDP (Democratic Progressive Party) legislator...  fake
5555  In Japan, if the new coronavirus test is negat...  fake
5556  We have a perfectly coordinated and fine tuned...  real
5557  Official death toll over 900 but we all know t...  fake
5558  The number of new cases from China's coronavir...  fake

[5559 rows x 2 columns]

In [4]:
# Misinformation Dataset
misin = pd.read_excel("Misinformation Dataset\Misinformation news.xlsx")

misin = misin[["tweet","label"]]
misin = misin.rename(columns={"tweet":"content"})

misin

content label
0     The CDC currently reports 99031 deaths. In gen...  real
1     States reported 1121 deaths a small rise from ...  real
2     Politically Correct Woman (Almost) Uses Pandem...  fake
3     #IndiaFightsCorona: We have 1524 #COVID testin...  real
4     Populous states can generate large case counts...  real
...                                                 ...   ...
6415  A tiger tested positive for COVID-19 please st...  fake
6416  ???Autopsies prove that COVID-19 is??� a blood...  fake
6417  _A post claims a COVID-19 vaccine has already ...  fake
6418  Aamir Khan Donate 250 Cr. In PM Relief Cares Fund  fake
6419  It has been 93 days since the last case of COV...  real

[6420 rows x 2 columns]

In [5]:
# FNIR Dataset
fnirT = pd.read_csv("FNIR Dataset\\FNIR true news.csv")
fnirF = pd.read_csv("FNIR Dataset\\FNIR fake news.csv")

fnirT = fnirT[["Text", "Label"]]
fnirF = fnirF[["Text", "Binary Label"]]

fnirT = fnirT.rename(columns={"Label":"label","Text":"content"})
fnirF = fnirF.rename(columns={"Binary Label":"label","Text":"content"})

fnir = pd.concat([fnirT,fnirF],ignore_index=True)

fnir["label"] = fnir["label"].replace(0,"fake")
fnir["label"] = fnir["label"].replace(1,"real")

fnir.reset_index(drop=True)


content label
0     Just in: Novel coronavirus named 'Covid-19': U...  real
1     WHO officially names #coronavirus as Covid-19....  real
2     The #UN #health agency announced that "COVID-1...  real
3     The Indian Embassy in Tokyo has said that one ...  real
4     Ground Zero | How Kerala used its experience i...  real
...                                                 ...   ...
7583   Bill Gates said that the COVID-19 vaccine wil...  fake
7584   COVID-19 vaccine candidates will insert micro...  fake
7585   An image claims that chroma screen panels are...  fake
7586   Alexandria Ocasio-Cortez tweeted, "It's vital...  fake
7587   The spread of COVID-19 began last October wit...  fake

[7588 rows x 2 columns]

In [ ]:
# Twitter collection
import tweepy
auth = tweepy.OAuthHandler("", "")
auth.set_access_token("", "")
api = tweepy.API(auth)

# list of covid misinformative accounts usernames in string
superspreaders = []
# list of credible covid accounts usernames in string
crediblesources = []
# initiate lists for fake news and real news
fakeNews = []
realNews = []

# collect tweet text
for i in superspreaders:
    for status in tweepy.Cursor(api.user_timeline, screen_name=str(i), count=200, tweet_mode="extended").items():
        fakeNews.append(status.full_text)

for i in crediblesources:
    for status in tweepy.Cursor(api.user_timeline, screen_name=str(i), count=200, tweet_mode="extended").items():
        realNews.append(status.full_text)

# construct dataframe from collected tweets and labelling
fakedf = pd.DataFrame(fakeNews, columns=["content"])
fakedf["label"] = 0

realdf = pd.DataFrame(realNews, columns=["content"])
realdf["label"] = 1

twitter = pd.concat([realdf,fakedf], ignore_index =True)


In [6]:
# Combine all datasets to create fake news df
df = pd.concat([fibvid,rumour,misin,fnir,twitter], ignore_index=True)
# clean df of duplicates and empty rows
df = df.dropna()
df = df.drop_duplicates()
# set labels: fake as 0, real as 1
df["label"] = df["label"].replace("fake",0)
df["label"] = df["label"].replace("real",1)

realdf = df[df["label"]==1]
fakedf = df[df["label"]==0]

print("real news count: ",realdf.shape[0])
print("fake news count: ",fakedf.shape[0])
print("total count: ", df.shape[0])

# Save fake news dataset
df.to_csv("raw fake news dataset.csv", index=False)

real news count:  9468
fake news count:  11630
total count:  21098


In [27]:
# categorisation data
fibvidCategorisation = fibvid1[(fibvid1.label == 2) | (fibvid1.label ==3)]

fibvidCategorisation["label"] = fibvidCategorisation["label"].replace(2,1) # non covid real
fibvidCategorisation["label"] = fibvidCategorisation["label"].replace(3,1) # non covid fake

fibvidCategorisation.reset_index(drop=True)
# fibvidCategorisation[fibvidCategorisation.label == "normal"]

# create covid vs non covid dataset
fibvidCategorisation = pd.concat([fibvid,fibvidCategorisation], ignore_index = True)
fibvidCategorisation["label"] = fibvidCategorisation["label"].replace("real",0)
fibvidCategorisation["label"] = fibvidCategorisation["label"].replace("fake",0)
fibvidCategorisation


content  label
0     the (covid 19) cases are going up but it's bec...      0
1     every election year has a disease  sars in 200...      0
2     if you paid rent in march or april they got to...      0
3              dr fauci: no reason to be wearing a mask      0
4       drinking boiled garlic water will cure covid 19      0
...                                                 ...    ...
3122  THANK GOD the Breitbart COVID video was remove...      1
3123  Today on US Google &amp; YouTube homepages we ...      1
3124  @kathygriffin God damn it! Lies like this duri...      1
3125  Straight-up punk-ass LIAR! #LiarInChief https:...      1
3126  “I would also like to point out that we've don...      1

[3127 rows x 2 columns]

In [28]:
# save news categorisation dataset
fibvidCategorisation.to_csv("raw categorisation dataset.csv")